In [2]:
import requests
from zipfile import ZipFile
from io import BytesIO
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import gzip
import shutil


In [ ]:
import os

def get_folder_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

def print_folder_sizes(base_directory):
    for root, dirs, files in os.walk(base_directory):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            size = get_folder_size(dir_path)
            print(f"Folder: {dir_path}, Size: {size / (1024 ** 2):.2f} MB")  # Convert bytes to MB

# Change 'data' to your base folder name if needed
print_folder_sizes('./data')


In [ ]:

datasets={
    "SGD": "data/SGD/",
    "MultiWOZ": "data/MultiWOZ/",
    "ABCD": "data/ABCD/",
    "BiToD": "data/BiToD/",
    "SMCalFlow": "data/SMCalFlow/",
    "TreeDST": "data/TreeDST/"
}

# Function to load JSON files from a directory
def load_json_files(directory):
    data = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".json"):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    data.append(json.load(f))
    return data

# Function to explore dataset
def explore_dataset(dataset_name, dataset_path):
    print(f"\nExploring {dataset_name}...")

    # Load data
    data = load_json_files(dataset_path)

    # Example: Analyze dialogue lengths
    dialogue_lengths = [len(dialogue['turns']) for dialogue in data]
    print(f"Average dialogue length: {sum(dialogue_lengths) / len(dialogue_lengths):.2f} turns")

    # Plot dialogue length distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(dialogue_lengths, bins=30, kde=True)
    plt.title(f"{dataset_name} Dialogue Length Distribution")
    plt.xlabel("Number of Turns")
    plt.ylabel("Frequency")
    plt.show()

    # Example: Analyze intents (if available)
    intents = []
    for dialogue in data:
        for turn in dialogue['turns']:
            if 'intent' in turn:
                intents.append(turn['intent'])

    if intents:
        intent_counts = Counter(intents)
        print(f"Top 5 intents in {dataset_name}: {intent_counts.most_common(5)}")

        # Plot intent distribution
        plt.figure(figsize=(12, 8))
        sns.barplot(x=[count for _, count in intent_counts.most_common(10)],
                    y=[intent for intent, _ in intent_counts.most_common(10)])
        plt.title(f"Top 10 Intents in {dataset_name}")
        plt.xlabel("Frequency")
        plt.ylabel("Intent")
        plt.show()

    # Additional analyses can be added here (e.g., slot distributions, confusion matrices)

# Explore each dataset
for name, info in datasets.items():
    explore_dataset(name, info)


In [ ]:
import pandas as pd
import os
from glob import glob

# Path to the folder containing JSON files
data_path = 'data/SGD/dstc8-schema-guided-dialogue-master/dev/'

# Load and process all JSON files
all_files = sorted(glob(os.path.join(data_path, 'dialogues_*.json')))

# Initialize a list to collect all services
all_services = []

for file in all_files:
    # Load each JSON file into a DataFrame
    df = pd.read_json(file)
    
    # Extract services and extend the list
    all_services.extend([service for services in df['services'] for service in services])

# Get unique services
unique_services = pd.Series(all_services).unique()

# Print unique services
print("Unique Services:")
print(unique_services)


Unique Services:
['Restaurants_2' 'Flights_3' 'RideSharing_1' 'RentalCars_1' 'Buses_1'
 'Alarm_1' 'Services_4' 'Weather_1' 'Homes_1' 'Banks_2' 'Travel_1'
 'Movies_2' 'Media_2' 'Hotels_4' 'Hotels_1' 'Music_1' 'Events_1']


In [ ]:
import json
def explore_json_structure(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if isinstance(data, list):
        print(f"The JSON file contains a list with {len(data)} items.")
        if len(data) > 0 and isinstance(data[0], dict):
            print("Each item in the list is a dictionary with the following keys:")
            for key in data[0].keys():
                print(f"- {key}")
    elif isinstance(data, dict):
        print("The JSON file contains a dictionary with the following keys:")
        for key in data.keys():
            print(f"- {key}")
    else:
        print("The JSON file contains an unknown structure.")

    return data

# Example usage:
dialogues_data = explore_json_structure('data/SGD/dstc8-schema-guided-dialogue-master/dev/dialogues_001.json')
schema_data = explore_json_structure('data/SGD/dstc8-schema-guided-dialogue-master/dev/schema.json')


The JSON file contains a list with 128 items.
Each item in the list is a dictionary with the following keys:
- dialogue_id
- services
- turns
The JSON file contains a list with 17 items.
Each item in the list is a dictionary with the following keys:
- service_name
- description
- slots
- intents


In [6]:
import pandas as pd

def load_json_to_dataframe(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if isinstance(data, list):
        df = pd.DataFrame(data)
    elif isinstance(data, dict):
        df = pd.DataFrame([data])
    else:
        raise ValueError("Unsupported JSON structure")
    
    return df

# Example usage:
dialogues_df = load_json_to_dataframe('data/SGD/dstc8-schema-guided-dialogue-master/dev/dialogues_001.json')
schema_df = load_json_to_dataframe('data/SGD/dstc8-schema-guided-dialogue-master/dev/schema.json')
print(dialogues_df.head())
print(schema_df.head())


  dialogue_id         services  \
0     1_00000  [Restaurants_2]   
1     1_00001  [Restaurants_2]   
2     1_00002  [Restaurants_2]   
3     1_00003  [Restaurants_2]   
4     1_00004  [Restaurants_2]   

                                               turns  
0  [{'frames': [{'actions': [{'act': 'INFORM', 'c...  
1  [{'frames': [{'actions': [{'act': 'INFORM_INTE...  
2  [{'frames': [{'actions': [{'act': 'INFORM', 'c...  
3  [{'frames': [{'actions': [{'act': 'INFORM', 'c...  
4  [{'frames': [{'actions': [{'act': 'INFORM', 'c...  
  service_name                                        description  \
0      Alarm_1   Manage alarms by getting and setting them easily   
1      Banks_2  Service to manage your bank accounts and finances   
2      Buses_1  Book bus journeys from the biggest bus network...   
3     Events_1  The comprehensive portal to find and reserve s...   
4    Flights_3  Find one way and round trip flights to your fa...   

                                               slo

In [8]:
def explore_services(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        services = json.load(f)
    
    print(f"Total number of services: {len(services)}")
    
    for service in services:
        print(f"\nService Name: {service['service_name']}")
        # print(f"Description: {service['description']}")
        # print(f"Number of Slots: {len(service['slots'])}")
        # print(f"Number of Intents: {len(service['intents'])}")
        
        # print("Slots:")
        # for slot in service['slots']:
        #     print(f"  - {slot['name']}: {slot['description']}")
        #     if slot['is_categorical']:
        #         print(f"    Possible Values: {slot['possible_values']}")
        
        # print("Intents:")
        # for intent in service['intents']:
        #     print(f"  - {intent['name']}: {intent['description']}")
        #     print(f"    Required Slots: {intent['required_slots']}")
        #     print(f"    Optional Slots: {intent['optional_slots']}")
        #     print(f"    Result Slots: {intent['result_slots']}")

# Example usage:
explore_services('data/SGD/dstc8-schema-guided-dialogue-master/dev/schema.json')


Total number of services: 17

Service Name: Alarm_1

Service Name: Banks_2

Service Name: Buses_1

Service Name: Events_1

Service Name: Flights_3

Service Name: Homes_1

Service Name: Hotels_1

Service Name: Hotels_4

Service Name: Media_2

Service Name: Movies_2

Service Name: Music_1

Service Name: RentalCars_1

Service Name: Restaurants_2

Service Name: RideSharing_1

Service Name: Services_4

Service Name: Travel_1

Service Name: Weather_1


In [9]:
import json
import csv

def json_to_csv(json_file, csv_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        dialogues = json.load(f)
    
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['dialogue_id', 'service', 'turn_index', 'speaker', 'utterance', 'act', 'slot', 'value']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for dialogue in dialogues:
            dialogue_id = dialogue['dialogue_id']
            services = dialogue['services']
            for service in services:
                for turn_index, turn in enumerate(dialogue['turns']):
                    speaker = turn.get('speaker', '')
                    utterance = turn.get('utterance', '')
                    frames = turn.get('frames', [])
                    for frame in frames:
                        frame_service = frame.get('service', '')
                        actions = frame.get('actions', [])
                        for action in actions:
                            act = action.get('act', '')
                            slot = action.get('slot', '')
                            values = action.get('values', [])
                            for value in values:
                                writer.writerow({
                                    'dialogue_id': dialogue_id,
                                    'service': frame_service,
                                    'turn_index': turn_index,
                                    'speaker': speaker,
                                    'utterance': utterance,
                                    'act': act,
                                    'slot': slot,
                                    'value': value
                                })
                        slots = frame.get('slots', [])
                        for slot in slots:
                            slot_name = slot.get('slot', '')
                            slot_value = slot.get('value', '')
                            writer.writerow({
                                'dialogue_id': dialogue_id,
                                'service': frame_service,
                                'turn_index': turn_index,
                                'speaker': speaker,
                                'utterance': utterance,
                                'act': '',
                                'slot': slot_name,
                                'value': slot_value
                            })

# Example usage:
json_to_csv('data/SGD/dstc8-schema-guided-dialogue-master/test/dialogues_001.json', 'dialogues_001.csv')